### Install the necessary library

In [1]:
#!pip install pyLDAvis

### Import the libraries

In [1]:
import pandas as pd
import spacy
from nltk.tokenize import word_tokenize
import re
import gensim

import nltk
from nltk.corpus import stopwords

### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [2]:
import wget
wget.download("https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json")

  7% [.....                                                                   ]  1761280 / 23237087

 24% [.................                                                       ]  5758976 / 23237087

 51% [....................................                                    ] 11853824 / 23237087

 85% [.............................................................           ] 19906560 / 23237087

100% [........................................................................] 23237087 / 23237087

'newsgroups.json'

### Load the dataset

In [3]:
df=pd.read_json('newsgroups.json')

In [4]:
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [5]:
df.target_names.value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: target_names, dtype: int64

### Preprocess the data

### Email Removal

In [6]:
def emailremoval(text):
    filtered_sentence = re.sub(r"\S*@\S*\s?", "", text)
    return filtered_sentence

In [7]:
df['parsed_content']=df['content'].apply(emailremoval)

In [8]:
df['parsed_content']

0        From: (where's my thing)\nSubject: WHAT car is...
1        From: (Guy Kuo)\nSubject: SI Clock Poll - Fina...
2        From: (Thomas E Willis)\nSubject: PB questions...
3        From: (Joe Green)\nSubject: Re: Weitek P9000 ?...
4        From: (Jonathan McDowell)\nSubject: Re: Shuttl...
                               ...                        
11309    From: (Jim Zisfein) \nSubject: Re: Migraines a...
11310    From: Subject: Screen Death: Mac Plus/512\nLin...
11311    From: (Will Estes)\nSubject: Mounting CPU Cool...
11312    From: (Steven Collins)\nSubject: Re: Sphere fr...
11313    From: (Kevin J. Gunning)\nSubject: stolen CBR9...
Name: parsed_content, Length: 11314, dtype: object

### Newline Removal

In [9]:
def newline_removal(text):
    filtered_sentence = re.sub(r"\n", "", text)
    return filtered_sentence

In [10]:
df['parsed_content']=df['parsed_content'].apply(newline_removal)

In [11]:
df['parsed_content']

0        From: (where's my thing)Subject: WHAT car is t...
1        From: (Guy Kuo)Subject: SI Clock Poll - Final ...
2        From: (Thomas E Willis)Subject: PB questions.....
3        From: (Joe Green)Subject: Re: Weitek P9000 ?Or...
4        From: (Jonathan McDowell)Subject: Re: Shuttle ...
                               ...                        
11309    From: (Jim Zisfein) Subject: Re: Migraines and...
11310    From: Subject: Screen Death: Mac Plus/512Lines...
11311    From: (Will Estes)Subject: Mounting CPU Cooler...
11312    From: (Steven Collins)Subject: Re: Sphere from...
11313    From: (Kevin J. Gunning)Subject: stolen CBR900...
Name: parsed_content, Length: 11314, dtype: object

### Single Quotes Removal

In [12]:
def singlequotes_removal(text):
    filtered_sentence = re.sub(r"'(\w+)'", "", text)
    return filtered_sentence

In [13]:
df['parsed_content']=df['parsed_content'].apply(singlequotes_removal)
df['parsed_content']

0        From: (where's my thing)Subject: WHAT car is t...
1        From: (Guy Kuo)Subject: SI Clock Poll - Final ...
2        From: (Thomas E Willis)Subject: PB questions.....
3        From: (Joe Green)Subject: Re: Weitek P9000 ?Or...
4        From: (Jonathan McDowell)Subject: Re: Shuttle ...
                               ...                        
11309    From: (Jim Zisfein) Subject: Re: Migraines and...
11310    From: Subject: Screen Death: Mac Plus/512Lines...
11311    From: (Will Estes)Subject: Mounting CPU Cooler...
11312    From: (Steven Collins)Subject: Re: Sphere from...
11313    From: (Kevin J. Gunning)Subject: stolen CBR900...
Name: parsed_content, Length: 11314, dtype: object

### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [14]:
def sent_to_words(col):
    col=pd.Series([gensim.utils.simple_preprocess(sent, deacc=False, min_len=2, max_len=15) 
                             for sent in col.values.tolist()]
                            )
    return col

In [15]:
df['parsed_content']=sent_to_words(df['parsed_content'])

In [16]:
df['parsed_content'] 

0        [from, where, my, thing, subject, what, car, i...
1        [from, guy, kuo, subject, si, clock, poll, fin...
2        [from, thomas, willis, subject, pb, questions,...
3        [from, joe, green, subject, re, weitek, organi...
4        [from, jonathan, mcdowell, subject, re, shuttl...
                               ...                        
11309    [from, jim, zisfein, subject, re, migraines, a...
11310    [from, subject, screen, death, mac, plus, line...
11311    [from, will, estes, subject, mounting, cpu, co...
11312    [from, steven, collins, subject, re, sphere, f...
11313    [from, kevin, gunning, subject, stolen, cbr, r...
Name: parsed_content, Length: 11314, dtype: object

### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

#### remove_stopwords( )

In [17]:
def remove_stopwords(texts):
    stop_words=stopwords.words('english')
    extended_list=['from','subject','re','edu','use']
    stop_words.extend(extended_list)
    filtered_text = [words for words in texts if not words in stop_words]
    return filtered_text

In [18]:
df['parsed_content']=df['parsed_content'].apply(remove_stopwords)
df['parsed_content'] 

0        [thing, car, nntp, posting, host, rac, wam, um...
1        [guy, kuo, si, clock, poll, final, callsummary...
2        [thomas, willis, pb, questions, organization, ...
3        [joe, green, weitek, organization, harris, com...
4        [jonathan, mcdowell, shuttle, launch, smithson...
                               ...                        
11309    [jim, zisfein, migraines, invention, factory, ...
11310    [screen, death, mac, plus, lines, organization...
11311    [estes, mounting, cpu, cooler, vertical, mail,...
11312    [steven, collins, sphere, points, organization...
11313    [kevin, gunning, stolen, cbr, rrorganization, ...
Name: parsed_content, Length: 11314, dtype: object

### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

#### make_bigrams( )

In [19]:
data_words = df['parsed_content'].values.tolist()

In [20]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)

In [21]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [22]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [23]:
data_words_bigrams = make_bigrams(data_words)

In [24]:
type(data_words_bigrams)

list

In [25]:
df['parsed_content']=pd.Series(data_words_bigrams)

In [26]:
df['parsed_content']

0        [thing, car, nntp_posting, host, rac_wam, umd_...
1        [guy_kuo, si, clock, poll, final, callsummary,...
2        [thomas, willis, pb, questions, organization, ...
3        [joe, green, weitek, organization, harris, com...
4        [jonathan, mcdowell, shuttle_launch, smithsoni...
                               ...                        
11309    [jim_zisfein, migraines, invention_factory, bb...
11310    [screen, death, mac, plus, lines, organization...
11311    [estes, mounting, cpu, cooler, vertical, mail,...
11312    [steven, collins, sphere_points, organization,...
11313    [kevin, gunning, stolen, cbr, rrorganization, ...
Name: parsed_content, Length: 11314, dtype: object

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [27]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#### lemmatizaton( )

In [28]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [30]:
print(data_lemmatized[:1])

[['thing', 'car', 'nntp_poste', 'host', 'university', 'maryland_college', 'parkline', 'wonder', 'enlighten', 'car', 'sawthe', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'yearsof', 'production', 'car', 'make', 'history', 'info', 'youhave', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [31]:
df['parsed_content']=pd.Series(data_lemmatized)
df['parsed_content']

0        [thing, car, nntp_poste, host, university, mar...
1        [clock, poll, final, callsummary, final, call,...
2        [question, organization, purdue_university, en...
3        [system, divisionline, write, write, article, ...
4        [observatory, usadistribution, sciline, articl...
                               ...                        
11309    [line, dn, consultation, cheap, scan, also, we...
11310    [screen, death, line, organization, problem, s...
11311    [este, mount, cpu, cool, vertical, mail, group...
11312    [sphere_point, organization, central, line, nn...
11313    [steal, rrorganization, california_institute, ...
Name: parsed_content, Length: 11314, dtype: object

### Create a Dictionary

In [32]:
from gensim.corpora.dictionary import Dictionary

texts=[]
for i in df['parsed_content'].values.tolist():
    texts.append(i)

In [33]:
dictionary = Dictionary(texts)

### Create Corpus

### Filter low-frequency words

In [34]:
dictionary.filter_extremes(no_below=10, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in texts]

### Create Index 2 word dictionary

In [35]:
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [36]:
from gensim.models import LdaModel
ldamodel = LdaModel(corpus, num_topics=20, id2word = id2word, passes=20, chunksize=100, alpha='auto')

### Print the Keyword in the 10 topics

In [47]:
ldamodel.top_topics(corpus)[0][0][1][1]

'article'

In [49]:
testing = [sent for sent in ldamodel.top_topics(corpus)]

[([(0.026177282, 'get'),
   (0.023263376, 'article'),
   (0.023056852, 'know'),
   (0.020875517, 'think'),
   (0.020290766, 'go'),
   (0.01825331, 'make'),
   (0.017657258, 'see'),
   (0.016594678, 'time'),
   (0.016238637, 'good'),
   (0.015897239, 'well'),
   (0.013561117, 'say'),
   (0.012265361, 'thing'),
   (0.012157979, 'want'),
   (0.012122683, 'take'),
   (0.01155504, 'way'),
   (0.011280233, 'organization'),
   (0.01099597, 'come'),
   (0.010774903, 'give'),
   (0.010362918, 'try'),
   (0.009866992, 'much')],
  -1.1888811280315095),
 ([(0.02173581, 'case'),
   (0.021393705, 'state'),
   (0.02099845, 'point'),
   (0.01964468, 'mean'),
   (0.019075774, 'question'),
   (0.017595908, 'group'),
   (0.017203854, 'however'),
   (0.015853534, 'right'),
   (0.015410534, 'law'),
   (0.015042252, 'issue'),
   (0.014480919, 'reason'),
   (0.013428943, 'consider'),
   (0.012296607, 'gun'),
   (0.011394918, 'cause'),
   (0.0101017, 'make'),
   (0.009974187, 'fact'),
   (0.009822491, 'differ

In [54]:
[[topic_score for topic_score in sent][0][0][1] for sent in ldamodel.top_topics(corpus)]

['get',
 'case',
 'people',
 'system',
 'high',
 'problem',
 'key',
 'program',
 'team',
 'space',
 'drive',
 'chip',
 'sale',
 'bit',
 'speak',
 'test',
 'armenian',
 'patient',
 'talk',
 'ax']

In [37]:
print(ldamodel.top_topics(corpus,topn=10))

[([(0.026177282, 'get'), (0.023263376, 'article'), (0.023056852, 'know'), (0.020875517, 'think'), (0.020290766, 'go'), (0.01825331, 'make'), (0.017657258, 'see'), (0.016594678, 'time'), (0.016238637, 'good'), (0.015897239, 'well')], -1.0346995472636469), ([(0.02173581, 'case'), (0.021393705, 'state'), (0.02099845, 'point'), (0.01964468, 'mean'), (0.019075774, 'question'), (0.017595908, 'group'), (0.017203854, 'however'), (0.015853534, 'right'), (0.015410534, 'law'), (0.015042252, 'issue')], -1.6282167145906923), ([(0.039375875, 'system'), (0.035113774, 'nntp_poste'), (0.03129419, 'line'), (0.031121679, 'host'), (0.023550356, 'need'), (0.02286703, 'use'), (0.021976534, 'thank'), (0.021411562, 'mail'), (0.01993114, 'new'), (0.018695222, 'send')], -1.7215634996296674), ([(0.05306641, 'people'), (0.037216783, 'say'), (0.026940912, 'evidence'), (0.025304522, 'believe'), (0.021857895, 'man'), (0.01817479, 'claim'), (0.014938672, 'child'), (0.014711189, 'life'), (0.014343399, 'sense'), (0.013

In [55]:
#15 topics
for idx in range(15):
    print("Topic #%s:" % idx, ldamodel.print_topic(idx, 4))

Topic #0: 0.120*"chip" + 0.080*"version" + 0.067*"internet" + 0.066*"pc"
Topic #1: 0.066*"problem" + 0.064*"run" + 0.058*"window" + 0.054*"card"
Topic #2: 0.045*"speak" + 0.038*"attack" + 0.035*"israeli" + 0.033*"soldier"
Topic #3: 0.086*"test" + 0.057*"fire" + 0.044*"notice" + 0.037*"inc_line"
Topic #4: 0.039*"system" + 0.035*"nntp_poste" + 0.031*"line" + 0.031*"host"
Topic #5: 0.026*"get" + 0.023*"article" + 0.023*"know" + 0.021*"think"
Topic #6: 0.067*"armenian" + 0.054*"village" + 0.053*"greek" + 0.049*"material"
Topic #7: 0.053*"people" + 0.037*"say" + 0.027*"evidence" + 0.025*"believe"
Topic #8: 0.101*"key" + 0.069*"government" + 0.062*"public" + 0.032*"video"
Topic #9: 0.060*"sale" + 0.052*"light" + 0.031*"board" + 0.031*"lead"
Topic #10: 0.068*"space" + 0.064*"power" + 0.052*"report" + 0.033*"news"
Topic #11: 0.088*"patient" + 0.065*"self" + 0.048*"disease" + 0.041*"treatment"
Topic #12: 0.083*"team" + 0.078*"game" + 0.059*"play" + 0.049*"win"
Topic #13: 0.135*"program" + 0.127

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [56]:
perplexity = ldamodel.log_perplexity(corpus)
perplexity

-11.557222751471857

### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [57]:
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4744592438336201


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [58]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(ldamodel, corpus, dictionary)
LDAvis_prepared

C:\Users\Murkasad\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
C:\Users\Murkasad\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.259608 -0.085482       1        1  32.786709
17     0.235415 -0.103776       2        1  11.270828
4      0.222153  0.197781       3        1   9.885049
7      0.220986 -0.182162       4        1   8.672871
14     0.125344  0.085049       5        1   7.067397
16    -0.103846 -0.064033       6        1   6.113632
1      0.076467  0.081824       7        1   3.337388
8      0.000965  0.270083       8        1   2.480808
10    -0.034886  0.052513       9        1   2.404709
12     0.003107 -0.118931      10        1   2.360382
2     -0.071372 -0.092432      11        1   1.880895
19    -0.059702 -0.055605      12        1   1.867902
13    -0.080054  0.157024      13        1   1.834486
18    -0.110280 -0.037655      14        1   1.633614
9     -0.114296 -0.076398      15        1   1.507882
0     -0.089330  0.111263      16        1   1.314658
3     -0.121325 -0.072221      17        1   1.167093
6     -0.071122  0.036305      18        1   0.951681
11    -0.143769 -0.061810      19        1   0.848911
15    -0.144061 -0.041340      20        1   0.613106, topic_info=         Term          Freq         Total Category  logprob  loglift
3155       ax  53070.000000  53070.000000  Default  30.0000  30.0000
121    people   6743.000000   6743.000000  Default  29.0000  29.0000
263       say   7933.000000   7933.000000  Default  28.0000  28.0000
336   problem   3730.000000   3730.000000  Default  27.0000  27.0000
808     drive   2686.000000   2686.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
5422       dn      0.074773      1.454664  Topic20 -11.3475   2.1263
3135     king      0.074772      1.454652  Topic20 -11.3475   2.1263
4042    print      0.074773      1.454705  Topic20 -11.3475   2.1263
2982   chance      0.074772      1.454696  Topic20 -11.3475   2.1263
3334   select      0.074772      1.454684  Topic20 -11.3476   2.1263

[839 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1677      8  0.992816      abuse
1108      4  0.998928     accept
767      12  0.993653   accident
525       9  0.996549   activity
151       3  0.999103    address
...     ...       ...        ...
660      17  0.642334  yesterday
518       1  0.407613      young
518       4  0.209629      young
518      11  0.381409      young
2033     17  0.990852       zone

[1021 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 18, 5, 8, 15, 17, 2, 9, 11, 13, 3, 20, 14, 19, 10, 1, 4, 7, 12, 16])

In [67]:
wp = ldamodel.show_topic(0)
topic_keywords = ", ".join([word for word, prop in wp])

In [68]:
topic_keywords

'chip, version, internet, pc, solution, device, scsi, family, controller, load'

In [70]:
for i, j in enumerate(ldamodel[corpus]):
    print(i,j)

0 [(1, 0.011781242), (4, 0.16355649), (5, 0.30644092), (7, 0.06112733), (13, 0.021552006), (14, 0.07776019), (17, 0.18895885), (19, 0.10166804)]
1 [(1, 0.05378344), (4, 0.15498465), (5, 0.25511253), (7, 0.02161201), (9, 0.14730948), (10, 0.1261774), (11, 0.031706393), (12, 0.029015752), (14, 0.04331278), (17, 0.05069681), (18, 0.03445762), (19, 0.018600393)]
2 [(1, 0.017064326), (2, 0.012003292), (3, 0.011091297), (4, 0.24630177), (5, 0.37451112), (7, 0.05071318), (11, 0.010191562), (12, 0.107554756), (14, 0.03247004), (17, 0.054127954), (18, 0.022511913), (19, 0.036207248)]
3 [(0, 0.051659502), (1, 0.034272872), (4, 0.16330925), (5, 0.47420186), (7, 0.03547246), (8, 0.010095963), (10, 0.010221335), (12, 0.03144115), (14, 0.045486324), (17, 0.08032376)]
4 [(0, 0.032663926), (1, 0.09300343), (2, 0.100882106), (3, 0.06666397), (4, 0.0730985), (5, 0.32431516), (7, 0.036131866), (9, 0.01994981), (10, 0.035622086), (13, 0.032907434), (14, 0.022161296), (17, 0.09702002), (18, 0.03445667)]
5 

923 [(1, 0.012145363), (2, 0.014953317), (4, 0.026563311), (5, 0.41030633), (7, 0.123150855), (9, 0.026183156), (12, 0.15439656), (14, 0.073442005), (17, 0.100753665), (18, 0.022580469)]
924 [(3, 0.030591667), (4, 0.08858778), (5, 0.5206953), (7, 0.016600197), (8, 0.014839662), (9, 0.049769804), (11, 0.03901471), (12, 0.028098049), (14, 0.04885648), (17, 0.030512698), (18, 0.023385622), (19, 0.08015246)]
925 [(0, 0.024846165), (1, 0.013972986), (3, 0.052257713), (4, 0.08542635), (5, 0.4661345), (7, 0.046642177), (10, 0.06634852), (14, 0.054553837), (17, 0.11926595)]
926 [(1, 0.018416109), (2, 0.036056846), (4, 0.06425057), (5, 0.41513), (7, 0.1717031), (8, 0.04865258), (12, 0.038018983), (14, 0.03868636), (17, 0.11922268)]
927 [(0, 0.06527108), (1, 0.012109911), (4, 0.24558637), (5, 0.2681618), (7, 0.027781187), (9, 0.040386394), (10, 0.042836882), (14, 0.054887533), (17, 0.12164814), (18, 0.024034472), (19, 0.049472786)]
928 [(0, 0.014929045), (2, 0.05242189), (4, 0.048861906), (5, 0.

1912 [(0, 0.01189712), (1, 0.088420846), (3, 0.020848734), (4, 0.08940116), (5, 0.42356554), (7, 0.03050868), (11, 0.063730486), (12, 0.01435701), (14, 0.10838559), (17, 0.09277456), (19, 0.02657323)]
1913 [(0, 0.13569261), (1, 0.06599738), (3, 0.021116193), (4, 0.10786153), (5, 0.34072018), (7, 0.029402163), (9, 0.011580773), (14, 0.0455162), (17, 0.0846189), (19, 0.10504402)]
1914 [(1, 0.019178536), (2, 0.012564887), (4, 0.06817497), (5, 0.45643905), (7, 0.045533195), (8, 0.012570035), (9, 0.010893133), (10, 0.013149996), (12, 0.014472786), (14, 0.0918637), (17, 0.1874061), (18, 0.011547746), (19, 0.011990598)]
1915 [(0, 0.07032689), (1, 0.113587774), (4, 0.10411258), (5, 0.30415896), (7, 0.0367244), (8, 0.027994988), (9, 0.016838497), (10, 0.0216356), (12, 0.017825248), (13, 0.028304348), (14, 0.07639705), (17, 0.09404035), (18, 0.053001128), (19, 0.01735065)]
1916 [(1, 0.018423725), (2, 0.011171859), (4, 0.03883072), (5, 0.430441), (7, 0.026203196), (9, 0.042251315), (12, 0.1639739

2860 [(1, 0.015997613), (2, 0.033777777), (4, 0.16413164), (5, 0.35850945), (7, 0.13470985), (8, 0.029030316), (9, 0.02757836), (11, 0.024478743), (12, 0.03615838), (13, 0.026195135), (14, 0.03335335), (17, 0.06693234)]
2861 [(0, 0.030030394), (1, 0.03551989), (3, 0.14722711), (4, 0.1032827), (5, 0.31323892), (7, 0.033475805), (9, 0.0155435), (10, 0.13875988), (11, 0.013669368), (14, 0.068585165), (17, 0.055364016), (18, 0.015826052)]
2862 [(1, 0.022111578), (2, 0.04394204), (3, 0.041291658), (4, 0.108203776), (5, 0.46357), (7, 0.07912359), (8, 0.015282767), (9, 0.012559057), (10, 0.015161072), (12, 0.015077781), (13, 0.010090537), (14, 0.044881735), (17, 0.07011899), (18, 0.013313784), (19, 0.015093161)]
2863 [(1, 0.016261004), (3, 0.027732084), (4, 0.20938148), (5, 0.40233505), (7, 0.034003515), (8, 0.010144537), (10, 0.01018213), (12, 0.010342475), (14, 0.102005064), (17, 0.05306169), (18, 0.06986971)]
2864 [(0, 0.010407077), (1, 0.024345815), (2, 0.014704247), (3, 0.011790908), (4,

3623 [(1, 0.019988265), (2, 0.047966547), (4, 0.025744798), (5, 0.3342215), (6, 0.13816191), (7, 0.2349748), (8, 0.013353514), (9, 0.011491215), (10, 0.012492329), (14, 0.05594298), (17, 0.056267027), (19, 0.0124168135)]
3624 [(0, 0.02929426), (1, 0.016176412), (2, 0.05448382), (3, 0.07495071), (4, 0.12905158), (5, 0.38281623), (7, 0.03207487), (8, 0.010548412), (9, 0.03156083), (10, 0.033816), (12, 0.040838055), (14, 0.03361299), (17, 0.088377446), (19, 0.010113643)]
3625 [(1, 0.010570998), (3, 0.046211846), (4, 0.038810104), (5, 0.39639443), (7, 0.045469496), (9, 0.034594897), (10, 0.021660272), (11, 0.017775146), (12, 0.21874371), (14, 0.03582367), (17, 0.09274866)]
3626 [(1, 0.016931958), (2, 0.103919566), (4, 0.058418993), (5, 0.4756293), (7, 0.077847086), (8, 0.011041093), (10, 0.049342334), (12, 0.011545823), (14, 0.043028034), (17, 0.059439942), (18, 0.010195041), (19, 0.034007713)]
3627 [(1, 0.022111814), (2, 0.014063708), (3, 0.01070896), (4, 0.09816172), (5, 0.3192603), (7, 

...

7083 [(0, 0.026337251), (1, 0.088186376), (4, 0.14808173), (5, 0.26676738), (7, 0.03318154), (8, 0.06982889), (12, 0.032101654), (13, 0.04689587), (14, 0.07475393), (17, 0.10733694), (18, 0.049015343)]
7084 [(1, 0.058272745), (2, 0.047220588), (3, 0.010710282), (4, 0.1640605), (5, 0.32849687), (7, 0.043672536), (8, 0.0151686175), (9, 0.038410652), (10, 0.015163107), (12, 0.015079805), (13, 0.0406758), (14, 0.09561642), (17, 0.0701284), (18, 0.013315572), (19, 0.013826218)]
7085 [(1, 0.010312354), (2, 0.037532117), (4, 0.036233764), (5, 0.5121236), (7, 0.08632423), (8, 0.017894918), (9, 0.039630793), (10, 0.053721346), (11, 0.014656244), (12, 0.017626574), (14, 0.04454148), (17, 0.09866174)]
7086 [(1, 0.024633009), (2, 0.013776044), (3, 0.011046609), (4, 0.19134842), (5, 0.301244), (7, 0.04504395), (8, 0.04963396), (9, 0.0129551785), (10, 0.015639262), (12, 0.015553344), (13, 0.07352299), (14, 0.0448523), (17, 0.14162649), (18, 0.013733709), (19, 0.014260393)]
7087 [(1, 0.014151704), (4

7967 [(0, 0.030620066), (1, 0.09035005), (3, 0.026449917), (4, 0.2806582), (5, 0.18690298), (7, 0.018477842), (8, 0.034565583), (9, 0.022368763), (10, 0.010891588), (13, 0.054245338), (14, 0.111814946), (17, 0.080563374), (18, 0.014491863), (19, 0.01029435)]
7968 [(1, 0.014261387), (4, 0.09156506), (5, 0.33324945), (7, 0.28672898), (14, 0.04777139), (17, 0.08925408), (18, 0.048043266)]
7969 [(0, 0.02730723), (1, 0.062476147), (3, 0.015053029), (4, 0.25183183), (5, 0.32475856), (7, 0.022180762), (10, 0.017266728), (11, 0.058438312), (13, 0.037372656), (14, 0.050684303), (17, 0.086079895), (18, 0.015998233)]
7970 [(1, 0.023646943), (2, 0.01377614), (3, 0.011046685), (4, 0.100920804), (5, 0.42396224), (7, 0.04504426), (8, 0.014873527), (9, 0.04450969), (10, 0.01563937), (12, 0.015553451), (13, 0.010408871), (14, 0.047521845), (17, 0.17245945), (18, 0.013733805), (19, 0.01577311)]
7971 [(1, 0.014791984), (4, 0.057779573), (5, 0.5226142), (7, 0.028476648), (14, 0.22745322), (17, 0.0529648)]

8882 [(1, 0.022454845), (2, 0.010804912), (4, 0.028602906), (5, 0.4372206), (7, 0.071794525), (8, 0.1569454), (9, 0.013877373), (14, 0.041403223), (17, 0.123059556), (19, 0.038361922)]
8883 [(0, 0.066033274), (1, 0.022643426), (4, 0.26161322), (5, 0.24244978), (7, 0.027044922), (9, 0.06016447), (10, 0.04519661), (11, 0.021885537), (14, 0.04647786), (17, 0.08529257), (18, 0.042958174), (19, 0.032248467)]
8884 [(1, 0.020254062), (2, 0.04024982), (3, 0.037807107), (4, 0.22238335), (5, 0.2951999), (7, 0.0959377), (8, 0.013207391), (9, 0.011504016), (10, 0.014104792), (12, 0.013811151), (14, 0.039828215), (17, 0.105946496), (18, 0.04021753), (19, 0.012663028)]
8885 [(0, 0.022689842), (1, 0.013956359), (3, 0.023387332), (4, 0.32485247), (5, 0.33592603), (7, 0.04411676), (9, 0.024638463), (11, 0.02150724), (14, 0.027160844), (17, 0.10002657)]
8886 [(1, 0.01393238), (2, 0.02651383), (3, 0.020862125), (4, 0.07199012), (5, 0.37545994), (7, 0.05035304), (8, 0.1326693), (9, 0.021805014), (14, 0.09

9727 [(1, 0.015039118), (4, 0.13607045), (5, 0.3345764), (7, 0.15274481), (8, 0.0135321375), (10, 0.011614464), (12, 0.010125805), (14, 0.09044538), (16, 0.023151306), (17, 0.08289071), (18, 0.07056849)]
9728 [(1, 0.011472838), (2, 0.033989202), (3, 0.018794222), (4, 0.09305545), (5, 0.30456057), (7, 0.14545605), (8, 0.025849685), (9, 0.033609714), (14, 0.0632197), (17, 0.18971896), (19, 0.039604783)]
9729 [(1, 0.062905855), (2, 0.042962898), (4, 0.03665382), (5, 0.25658435), (7, 0.19901434), (10, 0.046963535), (13, 0.02148912), (14, 0.088394724), (16, 0.010168465), (17, 0.19410168), (19, 0.0132359825)]
9730 [(0, 0.07449516), (1, 0.09578743), (4, 0.14203297), (5, 0.34527314), (7, 0.045654323), (8, 0.031119375), (9, 0.026260989), (13, 0.044280242), (14, 0.04910713), (17, 0.06516299), (18, 0.026725577)]
9731 [(0, 0.012027205), (1, 0.028135864), (2, 0.13377543), (3, 0.013626465), (4, 0.09735961), (5, 0.32950526), (7, 0.055563636), (8, 0.018347004), (9, 0.015980765), (10, 0.019291697), (12

10604 [(4, 0.14957038), (5, 0.31976035), (6, 0.014990637), (7, 0.1604762), (8, 0.043167975), (10, 0.03985417), (14, 0.11993732), (17, 0.099224806)]
10605 [(0, 0.06513001), (1, 0.028453672), (3, 0.06470246), (4, 0.19363397), (5, 0.31581774), (6, 0.013616341), (7, 0.018835772), (9, 0.019654427), (10, 0.018109955), (13, 0.016605968), (14, 0.13797198), (17, 0.036028862), (18, 0.0415451)]
10606 [(1, 0.02211201), (2, 0.04394438), (3, 0.010709055), (4, 0.2557152), (5, 0.25913382), (7, 0.043945994), (8, 0.01469117), (9, 0.043151084), (10, 0.015220378), (12, 0.015078076), (13, 0.07127739), (14, 0.045595426), (17, 0.10210904), (18, 0.013314046), (19, 0.013824634)]
10607 [(0, 0.023507254), (1, 0.029765444), (4, 0.21675694), (5, 0.3696065), (7, 0.052582677), (9, 0.02531982), (10, 0.0103894975), (11, 0.022262037), (13, 0.02387989), (14, 0.06547705), (17, 0.104639195)]
10608 [(1, 0.023134628), (2, 0.05032258), (3, 0.024766302), (4, 0.06717212), (5, 0.36135596), (7, 0.100782014), (9, 0.043706127), (1

In [ ]:
6. What is the Dominant topic and its percentage contribution in each document

In [71]:
def format_topics_sentences(texts, ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(data_lemmatized, ldamodel, corpus)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

C:\Users\Murkasad\AppData\Local\Temp\ipykernel_3884\2053116896.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
C:\Users\Murkasad\AppData\Local\Temp\ipykernel_3884\2053116896.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,5,0.3064,"get, article, know, think, go, make, see, time...","[thing, car, nntp_poste, host, university, mar..."
1,1,5,0.2551,"get, article, know, think, go, make, see, time...","[clock, poll, final, callsummary, final, call,..."
2,2,5,0.3745,"get, article, know, think, go, make, see, time...","[question, organization, purdue_university, en..."
3,3,5,0.4742,"get, article, know, think, go, make, see, time...","[system, divisionline, write, write, article, ..."
4,4,5,0.3243,"get, article, know, think, go, make, see, time...","[observatory, usadistribution, sciline, articl..."
5,5,17,0.4485,"case, state, point, mean, question, group, how...","[foxvog_dougla, amendment, idea, organization,..."
6,6,5,0.3078,"get, article, know, think, go, make, see, time...","[brian, man, delaney, brain, tumor, treatment,..."
7,7,0,0.2817,"chip, version, internet, pc, solution, device,...","[grubb_ide, las_cruce, nmlines_distribution, w..."
8,8,5,0.2899,"get, article, know, think, go, make, see, time...","[win, icon, help, please, organization, univer..."
9,9,5,0.2492,"get, article, know, think, go, make, see, time...","[stan_kerr, sigma_design, double, article, org..."
